In [1]:
import pandas

In [21]:
q_sheet_to_name = {
    "fy2019_q4": "FY 2019, 4th Q, July 1- Sept. 30, 2019",
    "fy2019_q3": "FY 2019, 3rd Q, April 1 - June 30, 2019",
    "fy2020_q1": "FY 2020, 1st Q, Oct. 1 - Dec. 31, 2019"
}

In [39]:
dfs = []
for sheet_name, quarter in q_sheet_to_name.items():
    df = pandas.read_excel('data/USCIS/Naturalizations by field office.xlsx', sheet_name=sheet_name, 
                           skiprows=8, usecols="A:I")
    df.columns = ['state', 'field_office', 'code', 'Field Office by State6', 'Field Office Code', 
                  'Applications Received2', 'Approved3', 'Denied4', 'Pending5']

    df['state'] = df[df.columns[0]].fillna(axis=0, method='ffill').map(lambda name: name.strip() if type(name) == str else float('nan'))
    df = df.rename(columns={'Unnamed: 1':"field_office"})
    df['field_office'] = df.field_office.map(lambda name: name.strip() if type(name) == str else float('nan'))
    df['quarter'] = quarter
    df['year'] = df.quarter.map(lambda q: q[-4:])
    df['fy'] = df.quarter.map(lambda q: q.split(',')[0])
    df.rename(columns={'Unnamed: 2':'code'}, inplace=True)
    df['code'] = df['code'].map(lambda code: code.strip() if type(code) == str else float('nan'))
    
    dfs.append(df.dropna(subset=['field_office']))
pandas.concat(dfs).sample(3)

,state,field_office,code,Field Office by State6,Field Office Code,Applications Received2,Approved3,Denied4,Pending5,quarter,year,fy
3,Alaska,Anchorage,ANC,334,391,101,403,12,12,"FY 2019, 4th Q, July 1- Sept. 30, 2019",2019,FY 2019
77,Nevada,Las Vegas,LVG,2238,3131,347,9039,D,D,"FY 2020, 1st Q, Oct. 1 - Dec. 31, 2019",2019,FY 2020
191,South Africa,Johannesburg,NaN,NaN,NaN,NaN,NaN,-,-,"FY 2019, 4th Q, July 1- Sept. 30, 2019",2019,FY 2019


In [44]:
new_data = pandas.concat(dfs)[['field_office',  'Applications Received2', 'Approved3', 
                    'Denied4', 'Pending5', 'quarter', 'state', 'code', 'fy']]
new_data.sample(3)

,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,fy
143,San Juan,66,1142,D,-,"FY 2019, 4th Q, July 1- Sept. 30, 2019",Puerto Rico,SAJ,FY 2019
169,Guatemala City,NaN,NaN,-,-,"FY 2019, 4th Q, July 1- Sept. 30, 2019",Guatemala,NaN,FY 2019
179,Nairobi,NaN,NaN,-,-,"FY 2020, 1st Q, Oct. 1 - Dec. 31, 2019",Kenya,NaN,FY 2020


In [45]:
master_df = pandas.read_csv('data/master_df.csv', index_col=0)

In [46]:
master_df.sample(3)

,field_office,Applications Received2,Approved3,Denied4,Pending5,quarter,state,code,year,fy
359,Athens,NaN,NaN,NaN,NaN,"FY 2017, 1st Q, Oct. 1-Dec. 31, 2016",Greece,NaN,2016,FY 2017
229,Greer,374,457,38,773,"FY 2017, 4th Q, July 1-Sept. 30, 2017",South Carolina,GRR,2017,FY 2017
117,New Orleans,"1,262",951,63,"2,155","FY 2016, 3rd Q, April 1-June 30, 2016",Louisiana,NOL,2016,FY 2016


In [50]:
new_master = pandas.concat([master_df, new_data])

In [51]:
len(master_df), len(new_data), len(new_master)

(2288, 324, 2612)

In [52]:
new_master.to_csv('data/master_df.csv')